# Chains
Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Chains allow us to combine multiple components together to create a single, coherent application.